# Simple Models

## Load Data

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [4]:
cat_cols = [x for x in train.columns if x.startswith('cat')]
cat_cols

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [21]:
x_train = train.drop(columns=['id','target'])
y_train = train['target']

## Submit

In [33]:
def submit(model, refit = True):
    if refit:
        model.fit(x_train, y_train)
    pred = model.predict_proba(test.drop(columns='id'))[:,1]
    submission = pd.DataFrame({'id':test['id'], 'target':pred})
    return submission

## Default Lightgbm Model

In [36]:
from categorical_transform import CategoricalTransform
from lightgbm.sklearn import LGBMClassifier
from sklearn.pipeline import Pipeline

In [24]:
lightgbm_pipe = Pipeline([("trans", CategoricalTransform(cat_cols)),
                          ("lgbm", LGBMClassifier(n_jobs = -2))])

In [26]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lightgbm_pipe, X=x_train, y=y_train, cv=5, return_train_score = True,
                         scoring='roc_auc')

In [27]:
scores

{'fit_time': array([1.98442268, 1.89280725, 1.93658161, 1.90987444, 1.91863799]),
 'score_time': array([0.21702623, 0.21839547, 0.2185061 , 0.21901035, 0.21766758]),
 'test_score': array([0.89273126, 0.8919831 , 0.89248019, 0.88876405, 0.89091073]),
 'train_score': array([0.90542568, 0.90538798, 0.90540394, 0.90588154, 0.90576755])}

In [28]:
scores['test_score'].mean()

0.8913738669734169

In [34]:
submission = submit(lightgbm_pipe)
submission.head()

,id,target
0,5,0.140532
1,6,0.368300
2,8,0.030049
3,9,0.177666
4,11,0.112767


In [35]:
submission.to_csv('submission_lightgbm_default.csv', index=False)

## Default CatBoost

In [46]:
from categorical_transform import IntegerCategoricalTransform
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

In [47]:
cbc = CatBoostClassifier(cat_features=cat_cols, thread_count=6)
catboost_pipe = Pipeline([("trans", IntegerCategoricalTransform(cat_cols)),
                          ("catboost", cbc)])

In [48]:
from sklearn.model_selection import cross_validate
scores = cross_validate(catboost_pipe, X=x_train, y=y_train, cv=5, return_train_score = True,
                         scoring='roc_auc')

Learning rate set to 0.106975
0:	learn: 0.6229801	total: 230ms	remaining: 3m 49s
1:	learn: 0.5669567	total: 448ms	remaining: 3m 43s
2:	learn: 0.5266791	total: 648ms	remaining: 3m 35s
3:	learn: 0.4943945	total: 841ms	remaining: 3m 29s
4:	learn: 0.4700231	total: 1.02s	remaining: 3m 24s
5:	learn: 0.4524109	total: 1.23s	remaining: 3m 23s
6:	learn: 0.4374665	total: 1.45s	remaining: 3m 25s
7:	learn: 0.4262120	total: 1.64s	remaining: 3m 23s
8:	learn: 0.4167499	total: 1.85s	remaining: 3m 23s
9:	learn: 0.4092016	total: 2.06s	remaining: 3m 23s
10:	learn: 0.4032674	total: 2.23s	remaining: 3m 20s
11:	learn: 0.3982690	total: 2.43s	remaining: 3m 19s
12:	learn: 0.3940876	total: 2.62s	remaining: 3m 18s
13:	learn: 0.3900131	total: 2.81s	remaining: 3m 17s
14:	learn: 0.3866464	total: 2.99s	remaining: 3m 16s
15:	learn: 0.3838592	total: 3.19s	remaining: 3m 16s
16:	learn: 0.3811879	total: 3.41s	remaining: 3m 17s
17:	learn: 0.3792233	total: 3.61s	remaining: 3m 17s
18:	learn: 0.3773823	total: 3.82s	remaining:

158:	learn: 0.3473513	total: 34.5s	remaining: 3m 2s
159:	learn: 0.3472465	total: 34.7s	remaining: 3m 2s
160:	learn: 0.3471649	total: 34.9s	remaining: 3m 2s
161:	learn: 0.3470827	total: 35.2s	remaining: 3m 1s
162:	learn: 0.3469770	total: 35.4s	remaining: 3m 1s
163:	learn: 0.3469431	total: 35.6s	remaining: 3m 1s
164:	learn: 0.3468628	total: 35.8s	remaining: 3m 1s
165:	learn: 0.3467932	total: 36s	remaining: 3m
166:	learn: 0.3467133	total: 36.2s	remaining: 3m
167:	learn: 0.3466549	total: 36.4s	remaining: 3m
168:	learn: 0.3465829	total: 36.7s	remaining: 3m
169:	learn: 0.3465132	total: 36.9s	remaining: 3m
170:	learn: 0.3464442	total: 37.1s	remaining: 2m 59s
171:	learn: 0.3463688	total: 37.3s	remaining: 2m 59s
172:	learn: 0.3463089	total: 37.6s	remaining: 2m 59s
173:	learn: 0.3462381	total: 37.8s	remaining: 2m 59s
174:	learn: 0.3461509	total: 38s	remaining: 2m 58s
175:	learn: 0.3461202	total: 38.2s	remaining: 2m 58s
176:	learn: 0.3460325	total: 38.4s	remaining: 2m 58s
177:	learn: 0.3459534	to

315:	learn: 0.3388966	total: 1m 9s	remaining: 2m 29s
316:	learn: 0.3388717	total: 1m 9s	remaining: 2m 29s
317:	learn: 0.3388298	total: 1m 9s	remaining: 2m 29s
318:	learn: 0.3387892	total: 1m 9s	remaining: 2m 28s
319:	learn: 0.3387503	total: 1m 9s	remaining: 2m 28s
320:	learn: 0.3386993	total: 1m 10s	remaining: 2m 28s
321:	learn: 0.3386745	total: 1m 10s	remaining: 2m 28s
322:	learn: 0.3386482	total: 1m 10s	remaining: 2m 28s
323:	learn: 0.3386197	total: 1m 10s	remaining: 2m 27s
324:	learn: 0.3385687	total: 1m 11s	remaining: 2m 27s
325:	learn: 0.3385252	total: 1m 11s	remaining: 2m 27s
326:	learn: 0.3385020	total: 1m 11s	remaining: 2m 27s
327:	learn: 0.3384442	total: 1m 11s	remaining: 2m 27s
328:	learn: 0.3384088	total: 1m 12s	remaining: 2m 26s
329:	learn: 0.3383758	total: 1m 12s	remaining: 2m 26s
330:	learn: 0.3383658	total: 1m 12s	remaining: 2m 26s
331:	learn: 0.3383160	total: 1m 12s	remaining: 2m 26s
332:	learn: 0.3382773	total: 1m 12s	remaining: 2m 25s
333:	learn: 0.3382495	total: 1m 1

468:	learn: 0.3335113	total: 1m 43s	remaining: 1m 57s
469:	learn: 0.3334735	total: 1m 43s	remaining: 1m 56s
470:	learn: 0.3334443	total: 1m 43s	remaining: 1m 56s
471:	learn: 0.3334199	total: 1m 44s	remaining: 1m 56s
472:	learn: 0.3333987	total: 1m 44s	remaining: 1m 56s
473:	learn: 0.3333418	total: 1m 44s	remaining: 1m 56s
474:	learn: 0.3333119	total: 1m 44s	remaining: 1m 55s
475:	learn: 0.3332610	total: 1m 45s	remaining: 1m 55s
476:	learn: 0.3332578	total: 1m 45s	remaining: 1m 55s
477:	learn: 0.3332230	total: 1m 45s	remaining: 1m 55s
478:	learn: 0.3331783	total: 1m 45s	remaining: 1m 55s
479:	learn: 0.3331547	total: 1m 46s	remaining: 1m 54s
480:	learn: 0.3331437	total: 1m 46s	remaining: 1m 54s
481:	learn: 0.3331308	total: 1m 46s	remaining: 1m 54s
482:	learn: 0.3331156	total: 1m 46s	remaining: 1m 54s
483:	learn: 0.3330686	total: 1m 46s	remaining: 1m 53s
484:	learn: 0.3330225	total: 1m 47s	remaining: 1m 53s
485:	learn: 0.3329996	total: 1m 47s	remaining: 1m 53s
486:	learn: 0.3329649	total:

621:	learn: 0.3290561	total: 2m 17s	remaining: 1m 23s
622:	learn: 0.3290301	total: 2m 17s	remaining: 1m 23s
623:	learn: 0.3290106	total: 2m 18s	remaining: 1m 23s
624:	learn: 0.3289907	total: 2m 18s	remaining: 1m 22s
625:	learn: 0.3289523	total: 2m 18s	remaining: 1m 22s
626:	learn: 0.3289211	total: 2m 18s	remaining: 1m 22s
627:	learn: 0.3288890	total: 2m 18s	remaining: 1m 22s
628:	learn: 0.3288676	total: 2m 19s	remaining: 1m 22s
629:	learn: 0.3288408	total: 2m 19s	remaining: 1m 21s
630:	learn: 0.3288184	total: 2m 19s	remaining: 1m 21s
631:	learn: 0.3287665	total: 2m 19s	remaining: 1m 21s
632:	learn: 0.3287380	total: 2m 20s	remaining: 1m 21s
633:	learn: 0.3287205	total: 2m 20s	remaining: 1m 20s
634:	learn: 0.3286784	total: 2m 20s	remaining: 1m 20s
635:	learn: 0.3286475	total: 2m 20s	remaining: 1m 20s
636:	learn: 0.3286096	total: 2m 20s	remaining: 1m 20s
637:	learn: 0.3285698	total: 2m 21s	remaining: 1m 20s
638:	learn: 0.3285527	total: 2m 21s	remaining: 1m 19s
639:	learn: 0.3285417	total:

...

353:	learn: 0.3375738	total: 1m 17s	remaining: 2m 21s
354:	learn: 0.3375331	total: 1m 17s	remaining: 2m 21s
355:	learn: 0.3375076	total: 1m 17s	remaining: 2m 20s
356:	learn: 0.3374698	total: 1m 18s	remaining: 2m 20s
357:	learn: 0.3374446	total: 1m 18s	remaining: 2m 20s
358:	learn: 0.3374097	total: 1m 18s	remaining: 2m 20s
359:	learn: 0.3373727	total: 1m 18s	remaining: 2m 20s
360:	learn: 0.3373275	total: 1m 19s	remaining: 2m 19s
361:	learn: 0.3372894	total: 1m 19s	remaining: 2m 19s
362:	learn: 0.3372612	total: 1m 19s	remaining: 2m 19s
363:	learn: 0.3372527	total: 1m 19s	remaining: 2m 19s
364:	learn: 0.3372076	total: 1m 19s	remaining: 2m 19s
365:	learn: 0.3371455	total: 1m 20s	remaining: 2m 18s
366:	learn: 0.3371090	total: 1m 20s	remaining: 2m 18s
367:	learn: 0.3370752	total: 1m 20s	remaining: 2m 18s
368:	learn: 0.3370223	total: 1m 20s	remaining: 2m 18s
369:	learn: 0.3370028	total: 1m 21s	remaining: 2m 18s
370:	learn: 0.3369842	total: 1m 21s	remaining: 2m 17s
371:	learn: 0.3369421	total:

506:	learn: 0.3324396	total: 1m 52s	remaining: 1m 49s
507:	learn: 0.3323955	total: 1m 52s	remaining: 1m 48s
508:	learn: 0.3323760	total: 1m 52s	remaining: 1m 48s
509:	learn: 0.3323736	total: 1m 52s	remaining: 1m 48s
510:	learn: 0.3323506	total: 1m 53s	remaining: 1m 48s
511:	learn: 0.3323324	total: 1m 53s	remaining: 1m 48s
512:	learn: 0.3323244	total: 1m 53s	remaining: 1m 47s
513:	learn: 0.3323112	total: 1m 53s	remaining: 1m 47s
514:	learn: 0.3322962	total: 1m 53s	remaining: 1m 47s
515:	learn: 0.3322519	total: 1m 54s	remaining: 1m 47s
516:	learn: 0.3322128	total: 1m 54s	remaining: 1m 46s
517:	learn: 0.3321986	total: 1m 54s	remaining: 1m 46s
518:	learn: 0.3321688	total: 1m 54s	remaining: 1m 46s
519:	learn: 0.3321483	total: 1m 55s	remaining: 1m 46s
520:	learn: 0.3320995	total: 1m 55s	remaining: 1m 46s
521:	learn: 0.3320502	total: 1m 55s	remaining: 1m 45s
522:	learn: 0.3320109	total: 1m 55s	remaining: 1m 45s
523:	learn: 0.3319828	total: 1m 56s	remaining: 1m 45s
524:	learn: 0.3319719	total:

659:	learn: 0.3281165	total: 2m 26s	remaining: 1m 15s
660:	learn: 0.3280884	total: 2m 27s	remaining: 1m 15s
661:	learn: 0.3280463	total: 2m 27s	remaining: 1m 15s
662:	learn: 0.3280162	total: 2m 27s	remaining: 1m 14s
663:	learn: 0.3280059	total: 2m 27s	remaining: 1m 14s
664:	learn: 0.3279801	total: 2m 27s	remaining: 1m 14s
665:	learn: 0.3279523	total: 2m 28s	remaining: 1m 14s
666:	learn: 0.3279333	total: 2m 28s	remaining: 1m 14s
667:	learn: 0.3279160	total: 2m 28s	remaining: 1m 13s
668:	learn: 0.3278929	total: 2m 28s	remaining: 1m 13s
669:	learn: 0.3278560	total: 2m 29s	remaining: 1m 13s
670:	learn: 0.3278195	total: 2m 29s	remaining: 1m 13s
671:	learn: 0.3278143	total: 2m 29s	remaining: 1m 12s
672:	learn: 0.3277940	total: 2m 29s	remaining: 1m 12s
673:	learn: 0.3277710	total: 2m 29s	remaining: 1m 12s
674:	learn: 0.3277412	total: 2m 30s	remaining: 1m 12s
675:	learn: 0.3276845	total: 2m 30s	remaining: 1m 12s
676:	learn: 0.3276542	total: 2m 30s	remaining: 1m 11s
677:	learn: 0.3276420	total:

814:	learn: 0.3242984	total: 3m 1s	remaining: 41.2s
815:	learn: 0.3242765	total: 3m 1s	remaining: 41s
816:	learn: 0.3242516	total: 3m 2s	remaining: 40.8s
817:	learn: 0.3242130	total: 3m 2s	remaining: 40.6s
818:	learn: 0.3241714	total: 3m 2s	remaining: 40.3s
819:	learn: 0.3241470	total: 3m 2s	remaining: 40.1s
820:	learn: 0.3241370	total: 3m 2s	remaining: 39.9s
821:	learn: 0.3241062	total: 3m 3s	remaining: 39.7s
822:	learn: 0.3240829	total: 3m 3s	remaining: 39.5s
823:	learn: 0.3240620	total: 3m 3s	remaining: 39.2s
824:	learn: 0.3240287	total: 3m 3s	remaining: 39s
825:	learn: 0.3239966	total: 3m 4s	remaining: 38.8s
826:	learn: 0.3239671	total: 3m 4s	remaining: 38.6s
827:	learn: 0.3239516	total: 3m 4s	remaining: 38.3s
828:	learn: 0.3239083	total: 3m 4s	remaining: 38.1s
829:	learn: 0.3238922	total: 3m 5s	remaining: 37.9s
830:	learn: 0.3238803	total: 3m 5s	remaining: 37.7s
831:	learn: 0.3238703	total: 3m 5s	remaining: 37.5s
832:	learn: 0.3238357	total: 3m 5s	remaining: 37.2s
833:	learn: 0.32

970:	learn: 0.3203311	total: 3m 37s	remaining: 6.48s
971:	learn: 0.3203180	total: 3m 37s	remaining: 6.26s
972:	learn: 0.3202956	total: 3m 37s	remaining: 6.03s
973:	learn: 0.3202543	total: 3m 37s	remaining: 5.81s
974:	learn: 0.3202264	total: 3m 37s	remaining: 5.59s
975:	learn: 0.3202060	total: 3m 38s	remaining: 5.36s
976:	learn: 0.3201643	total: 3m 38s	remaining: 5.14s
977:	learn: 0.3201359	total: 3m 38s	remaining: 4.92s
978:	learn: 0.3201068	total: 3m 38s	remaining: 4.69s
979:	learn: 0.3200848	total: 3m 39s	remaining: 4.47s
980:	learn: 0.3200739	total: 3m 39s	remaining: 4.25s
981:	learn: 0.3200581	total: 3m 39s	remaining: 4.02s
982:	learn: 0.3200403	total: 3m 39s	remaining: 3.8s
983:	learn: 0.3200196	total: 3m 39s	remaining: 3.58s
984:	learn: 0.3200062	total: 3m 40s	remaining: 3.35s
985:	learn: 0.3199877	total: 3m 40s	remaining: 3.13s
986:	learn: 0.3199860	total: 3m 40s	remaining: 2.91s
987:	learn: 0.3199756	total: 3m 40s	remaining: 2.68s
988:	learn: 0.3199659	total: 3m 41s	remaining: 

In [52]:
scores

{'fit_time': array([231.92791295, 232.32997346, 231.22905755, 233.41568851,
        232.94989324]),
 'score_time': array([2.14417529, 2.17010641, 2.15229368, 2.15843081, 2.15452361]),
 'test_score': array([0.89528238, 0.89507526, 0.89521648, 0.89185385, 0.89425513]),
 'train_score': array([0.91086488, 0.91153714, 0.91108545, 0.91244879, 0.91151574])}

In [53]:
scores['test_score'].mean()

0.8943366207411788

In [51]:
submission = submit(catboost_pipe)
submission.head()

Learning rate set to 0.11767
0:	learn: 0.6139339	total: 329ms	remaining: 5m 28s
1:	learn: 0.5567925	total: 619ms	remaining: 5m 8s
2:	learn: 0.5154786	total: 916ms	remaining: 5m 4s
3:	learn: 0.4828273	total: 1.17s	remaining: 4m 50s
4:	learn: 0.4588615	total: 1.41s	remaining: 4m 40s
5:	learn: 0.4406216	total: 1.65s	remaining: 4m 34s
6:	learn: 0.4269822	total: 1.88s	remaining: 4m 26s
7:	learn: 0.4167466	total: 2.13s	remaining: 4m 24s
8:	learn: 0.4081431	total: 2.39s	remaining: 4m 23s
9:	learn: 0.4010793	total: 2.64s	remaining: 4m 21s
10:	learn: 0.3956042	total: 2.87s	remaining: 4m 17s
11:	learn: 0.3911239	total: 3.16s	remaining: 4m 19s
12:	learn: 0.3877442	total: 3.45s	remaining: 4m 21s
13:	learn: 0.3842181	total: 3.74s	remaining: 4m 23s
14:	learn: 0.3813668	total: 4s	remaining: 4m 22s
15:	learn: 0.3791249	total: 4.27s	remaining: 4m 22s
16:	learn: 0.3769691	total: 4.54s	remaining: 4m 22s
17:	learn: 0.3751698	total: 4.81s	remaining: 4m 22s
18:	learn: 0.3735114	total: 5.07s	remaining: 4m 21

158:	learn: 0.3464729	total: 43.8s	remaining: 3m 51s
159:	learn: 0.3464194	total: 44.1s	remaining: 3m 51s
160:	learn: 0.3463399	total: 44.4s	remaining: 3m 51s
161:	learn: 0.3462454	total: 44.6s	remaining: 3m 50s
162:	learn: 0.3461603	total: 44.9s	remaining: 3m 50s
163:	learn: 0.3460737	total: 45.2s	remaining: 3m 50s
164:	learn: 0.3460254	total: 45.4s	remaining: 3m 49s
165:	learn: 0.3459591	total: 45.7s	remaining: 3m 49s
166:	learn: 0.3458915	total: 45.9s	remaining: 3m 49s
167:	learn: 0.3458286	total: 46.2s	remaining: 3m 48s
168:	learn: 0.3457772	total: 46.5s	remaining: 3m 48s
169:	learn: 0.3457172	total: 46.8s	remaining: 3m 48s
170:	learn: 0.3456445	total: 47.1s	remaining: 3m 48s
171:	learn: 0.3455899	total: 47.4s	remaining: 3m 47s
172:	learn: 0.3455163	total: 47.6s	remaining: 3m 47s
173:	learn: 0.3454366	total: 47.9s	remaining: 3m 47s
174:	learn: 0.3453928	total: 48.2s	remaining: 3m 47s
175:	learn: 0.3453480	total: 48.5s	remaining: 3m 47s
176:	learn: 0.3452393	total: 48.8s	remaining: 

312:	learn: 0.3387062	total: 1m 28s	remaining: 3m 14s
313:	learn: 0.3386585	total: 1m 28s	remaining: 3m 14s
314:	learn: 0.3386060	total: 1m 29s	remaining: 3m 13s
315:	learn: 0.3385702	total: 1m 29s	remaining: 3m 13s
316:	learn: 0.3385564	total: 1m 29s	remaining: 3m 13s
317:	learn: 0.3384815	total: 1m 30s	remaining: 3m 13s
318:	learn: 0.3384352	total: 1m 30s	remaining: 3m 12s
319:	learn: 0.3384091	total: 1m 30s	remaining: 3m 12s
320:	learn: 0.3384024	total: 1m 30s	remaining: 3m 12s
321:	learn: 0.3383668	total: 1m 31s	remaining: 3m 11s
322:	learn: 0.3383482	total: 1m 31s	remaining: 3m 11s
323:	learn: 0.3383204	total: 1m 31s	remaining: 3m 11s
324:	learn: 0.3382656	total: 1m 32s	remaining: 3m 11s
325:	learn: 0.3382148	total: 1m 32s	remaining: 3m 10s
326:	learn: 0.3381692	total: 1m 32s	remaining: 3m 10s
327:	learn: 0.3381036	total: 1m 32s	remaining: 3m 10s
328:	learn: 0.3380933	total: 1m 33s	remaining: 3m 10s
329:	learn: 0.3380678	total: 1m 33s	remaining: 3m 9s
330:	learn: 0.3380271	total: 

466:	learn: 0.3334386	total: 2m 13s	remaining: 2m 32s
467:	learn: 0.3333995	total: 2m 13s	remaining: 2m 31s
468:	learn: 0.3333756	total: 2m 13s	remaining: 2m 31s
469:	learn: 0.3333504	total: 2m 14s	remaining: 2m 31s
470:	learn: 0.3333279	total: 2m 14s	remaining: 2m 31s
471:	learn: 0.3333040	total: 2m 14s	remaining: 2m 30s
472:	learn: 0.3332709	total: 2m 15s	remaining: 2m 30s
473:	learn: 0.3332394	total: 2m 15s	remaining: 2m 30s
474:	learn: 0.3332190	total: 2m 15s	remaining: 2m 29s
475:	learn: 0.3332044	total: 2m 15s	remaining: 2m 29s
476:	learn: 0.3331749	total: 2m 16s	remaining: 2m 29s
477:	learn: 0.3331543	total: 2m 16s	remaining: 2m 29s
478:	learn: 0.3331179	total: 2m 16s	remaining: 2m 28s
479:	learn: 0.3330908	total: 2m 17s	remaining: 2m 28s
480:	learn: 0.3330657	total: 2m 17s	remaining: 2m 28s
481:	learn: 0.3330401	total: 2m 17s	remaining: 2m 27s
482:	learn: 0.3329994	total: 2m 17s	remaining: 2m 27s
483:	learn: 0.3329717	total: 2m 18s	remaining: 2m 27s
484:	learn: 0.3329543	total:

619:	learn: 0.3292645	total: 2m 57s	remaining: 1m 48s
620:	learn: 0.3292324	total: 2m 57s	remaining: 1m 48s
621:	learn: 0.3292110	total: 2m 58s	remaining: 1m 48s
622:	learn: 0.3291816	total: 2m 58s	remaining: 1m 47s
623:	learn: 0.3291460	total: 2m 58s	remaining: 1m 47s
624:	learn: 0.3291253	total: 2m 59s	remaining: 1m 47s
625:	learn: 0.3291133	total: 2m 59s	remaining: 1m 47s
626:	learn: 0.3291046	total: 2m 59s	remaining: 1m 46s
627:	learn: 0.3290966	total: 2m 59s	remaining: 1m 46s
628:	learn: 0.3290463	total: 3m	remaining: 1m 46s
629:	learn: 0.3290121	total: 3m	remaining: 1m 45s
630:	learn: 0.3289780	total: 3m	remaining: 1m 45s
631:	learn: 0.3289421	total: 3m 1s	remaining: 1m 45s
632:	learn: 0.3289134	total: 3m 1s	remaining: 1m 45s
633:	learn: 0.3288842	total: 3m 1s	remaining: 1m 44s
634:	learn: 0.3288687	total: 3m 1s	remaining: 1m 44s
635:	learn: 0.3288551	total: 3m 2s	remaining: 1m 44s
636:	learn: 0.3288180	total: 3m 2s	remaining: 1m 43s
637:	learn: 0.3287717	total: 3m 2s	remaining: 

772:	learn: 0.3255329	total: 3m 41s	remaining: 1m 5s
773:	learn: 0.3255193	total: 3m 42s	remaining: 1m 4s
774:	learn: 0.3254784	total: 3m 42s	remaining: 1m 4s
775:	learn: 0.3254616	total: 3m 42s	remaining: 1m 4s
776:	learn: 0.3254436	total: 3m 43s	remaining: 1m 4s
777:	learn: 0.3254106	total: 3m 43s	remaining: 1m 3s
778:	learn: 0.3253873	total: 3m 43s	remaining: 1m 3s
779:	learn: 0.3253469	total: 3m 43s	remaining: 1m 3s
780:	learn: 0.3253368	total: 3m 44s	remaining: 1m 2s
781:	learn: 0.3253022	total: 3m 44s	remaining: 1m 2s
782:	learn: 0.3252735	total: 3m 44s	remaining: 1m 2s
783:	learn: 0.3252558	total: 3m 45s	remaining: 1m 1s
784:	learn: 0.3252238	total: 3m 45s	remaining: 1m 1s
785:	learn: 0.3251898	total: 3m 45s	remaining: 1m 1s
786:	learn: 0.3251711	total: 3m 45s	remaining: 1m 1s
787:	learn: 0.3251384	total: 3m 46s	remaining: 1m
788:	learn: 0.3251233	total: 3m 46s	remaining: 1m
789:	learn: 0.3251210	total: 3m 46s	remaining: 1m
790:	learn: 0.3250995	total: 3m 47s	remaining: 60s
791:

929:	learn: 0.3220854	total: 4m 27s	remaining: 20.1s
930:	learn: 0.3220593	total: 4m 27s	remaining: 19.8s
931:	learn: 0.3220345	total: 4m 27s	remaining: 19.5s
932:	learn: 0.3219987	total: 4m 28s	remaining: 19.3s
933:	learn: 0.3219549	total: 4m 28s	remaining: 19s
934:	learn: 0.3219481	total: 4m 28s	remaining: 18.7s
935:	learn: 0.3219162	total: 4m 29s	remaining: 18.4s
936:	learn: 0.3218996	total: 4m 29s	remaining: 18.1s
937:	learn: 0.3218753	total: 4m 29s	remaining: 17.8s
938:	learn: 0.3218541	total: 4m 29s	remaining: 17.5s
939:	learn: 0.3218311	total: 4m 30s	remaining: 17.3s
940:	learn: 0.3218154	total: 4m 30s	remaining: 17s
941:	learn: 0.3217943	total: 4m 30s	remaining: 16.7s
942:	learn: 0.3217678	total: 4m 31s	remaining: 16.4s
943:	learn: 0.3217616	total: 4m 31s	remaining: 16.1s
944:	learn: 0.3217476	total: 4m 31s	remaining: 15.8s
945:	learn: 0.3217436	total: 4m 32s	remaining: 15.5s
946:	learn: 0.3217314	total: 4m 32s	remaining: 15.2s
947:	learn: 0.3217224	total: 4m 32s	remaining: 15s

,id,target
0,5,0.140137
1,6,0.395102
2,8,0.015622
3,9,0.217600
4,11,0.140418


In [54]:
submission.to_csv('submission_catboost_default.csv', index=False)